Daniel Ross  
October, 2022  
HyperMaze

In [99]:
import pandas as pd
import numpy as np
import random

import igraph as ig

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection

In [101]:
m1 = HyperMaze(3)

In [102]:
m1.buildMaze()

In [103]:
m1.plot()

In [100]:
class HyperMaze:
    """Hyper Maze Class"""
    
    def __init__(self, dim):
        self.dim = dim
        self.g = ig.Graph.Lattice([dim,dim,dim], circular=False)
        self.g_vertex_df = pd.DataFrame()
        self.g_edge_df = pd.DataFrame()
        self.wall_df = pd.DataFrame()
      
    def buildMaze(self):
        self.g.es["weight"] = [random.randint(1, 77) for _ in self.g.es]
        
        mst = self.g.spanning_tree(weights=self.g.es["weight"], return_tree=False)
    
        self.g_vertex_df = self.g.get_vertex_dataframe()
        self.g_vertex_df.index.names = ['vertexid']
        self.g_vertex_df['vertex'] = self.g_vertex_df.index
        
        def get_x(v, d): return v % d
    
        def get_y(v, d): return (v // d) % d

        def get_z(v, d): 
            for i in range(d): 
                if v < ((d**2) * (i+1)): 
                    return i
                
        def get_cartesian_coords(v, d):
            x = get_x(v, d)
            y = get_y(v, d)
            z = get_z(v, d)
            return (x, y, z)
                
        self.g_vertex_df['x'] = self.g_vertex_df['vertex'].apply(get_x, d=self.dim)
        self.g_vertex_df['y'] = self.g_vertex_df['vertex'].apply(get_y, d=self.dim)
        self.g_vertex_df['z'] = self.g_vertex_df['vertex'].apply(get_z, d=self.dim)
        
        self.g_edge_df = pd.DataFrame(self.g.get_edge_dataframe())
        self.g_edge_df.index.names = ['edge']
        self.g_edge_df.columns = ['a', 'b', 'w']
        self.g_edge_df["ab"] = list(zip(self.g_edge_df['a'], self.g_edge_df['b']))
        
        self.g_edge_df['mst'] = self.g_edge_df.index.isin(mst)

        mst_edges = self.g_edge_df.loc[self.g_edge_df['mst'], ['a', 'b']]
        mst_edges = list(zip(mst_edges['a'], mst_edges['b']))
        
        sp = ig.Graph([self.dim, self.dim, self.dim], edges=mst_edges)
        shortest_path = sp.get_shortest_paths(0,(self.dim**3)-1, output="epath")
        
        sp_edge_df = sp.get_edge_dataframe()
        sp_edge_df['path'] = sp_edge_df.index.isin(shortest_path[0])
        sp_edge_df = sp_edge_df.loc[sp_edge_df['path'],:]

        sp_edge_df['ab'] = list(zip(sp_edge_df['source'], sp_edge_df['target']))
        sp_verts = list(sp_edge_df['ab'])

        def in_sp(v): return v in sp_verts

        self.g_edge_df['path'] = self.g_edge_df['ab'].apply(in_sp)

        self.g_edge_df['a_xyz'] = self.g_edge_df['a'].apply(get_cartesian_coords, d=self.dim)
        self.g_edge_df['b_xyz'] = self.g_edge_df['b'].apply(get_cartesian_coords, d=self.dim)
        self.g_edge_df['ab_xyz'] = list(zip(self.g_edge_df['a_xyz'], self.g_edge_df['b_xyz']))
        
        def wall_verts(v):
            if v[0][0] != v[1][0]:
                x = [(v[0][0] + v[1][0])/2] * 4
                y = [v[0][1] - .5, v[1][1] + .5]
                y=y+y[::-1]
                z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
            elif v[0][1] != v[1][1]:
                x= [v[0][0] - .5, v[1][0] + .5]
                x=x+x[::-1]
                y = [(v[0][1] + v[1][1])/2]*4
                z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
            else:
                x= [v[0][0] - .5, v[1][0] + .5]
                x=x+x[::-1]
                y = [v[0][1] - .5]*2 + [v[1][1] + .5]*2     
                z = [(v[0][2] + v[1][2])/2]*4
            #coord = np.array(np.meshgrid(x,y,z)).T.reshape(-1,3)
            #coord = list(map(tuple,coord))
            return list(zip(x,y,z))
        
        self.g_edge_df['wall'] = self.g_edge_df['ab_xyz'].apply(wall_verts)

        self.wall_df = self.g_edge_df.loc[self.g_edge_df['mst']==False, 'wall']
        
    def plot(self, in_notebook=False,
                   title="default",
                   mst_color="purple", sp_color="forestgreen", 
                   v_color="purple", vgoals_color="forestgreen", 
                   v_size=10, vgoals_size=50,
                   wall_face_color = "grey",
                   wall_edge_color = "black"):
        
        if title=="default":
            title = "HyperMaze" + str(self.dim) + "x" + str(self.dim)
            
        self.g_edge_df['color'] = "white"
        self.g_edge_df.loc[self.g_edge_df['mst'], 'color'] = mst_color
        self.g_edge_df.loc[self.g_edge_df['path'], 'color'] = sp_color

        self.g_vertex_df['vcolor'] = v_color
        self.g_vertex_df.loc[[0,len(self.g_vertex_df)-1], 'vcolor'] = vgoals_color
        self.g_vertex_df['vsize'] = v_size
        self.g_vertex_df.loc[[0,len(self.g_vertex_df)-1], 'vsize'] = vgoals_size

        wall_color = wall_face_color
        wall_edge_color = wall_edge_color
        
        if in_notebook:
            %matplotlib notebook
        else:
            %matplotlib qt
        
        fig = plt.figure(title)
        ax = fig.add_subplot(projection='3d')

        ax.set_axis_off()
        plt.tight_layout()

        ax.scatter(self.g_vertex_df['x'], self.g_vertex_df['y'], self.g_vertex_df['z'], 
                   c=self.g_vertex_df['vcolor'], 
                   s=self.g_vertex_df['vsize'])

        ax.add_collection3d(Line3DCollection(np.array(self.g_edge_df.loc[self.g_edge_df['mst'],'ab_xyz']),
                                             colors=self.g_edge_df.loc[self.g_edge_df['mst'],'color']))

        walls = ax.add_collection3d(Poly3DCollection(list(self.wall_df), linewidths=1, alpha=0.2))
        walls.set_facecolor(wall_color)
        walls.set_edgecolor(wall_edge_color)
        ax.add_collection3d(walls)

        ax.set_xlim([-(self.dim),(self.dim)])
        ax.set_ylim([-(self.dim),(self.dim)])
        ax.set_zlim([-(self.dim),(self.dim)])

In [39]:
dim = 4

In [40]:
g = ig.Graph.Lattice([dim,dim,dim], circular=False)

In [41]:
g.es["weight"] = [random.randint(1, 77) for _ in g.es]

In [42]:
mst = g.spanning_tree(weights=g.es["weight"], return_tree=False)

In [43]:
g_vertex_df = g.get_vertex_dataframe()
g_vertex_df.index.names = ['vertexid']
g_vertex_df['vertex'] = g_vertex_df.index

In [44]:
def get_x(v):
    return v % dim

def get_y(v):
    return (v // dim) % dim

def get_z(v):
    for i in range(dim):
        if v < ((dim**2) * (i+1)):
            return i

def get_cartesian_coords(v):
    x = get_x(v)
    y = get_y(v)
    z = get_z(v)
    return (x, y, z)

In [45]:
g_vertex_df['x'] = g_vertex_df['vertex'].apply(get_x)
g_vertex_df['y'] = g_vertex_df['vertex'].apply(get_y)
g_vertex_df['z'] = g_vertex_df['vertex'].apply(get_z)
g_vertex_df['xyz'] = g_vertex_df['vertex'].apply(get_cartesian_coords)

In [46]:
g_edge_df = pd.DataFrame(g.get_edge_dataframe())
g_edge_df.index.names = ['edge']
g_edge_df.columns = ['a', 'b', 'w']
g_edge_df["ab"] = list(zip(g_edge_df['a'], g_edge_df['b']))

In [47]:
g_edge_df['mst'] = g_edge_df.index.isin(mst)

mst_edges = g_edge_df.loc[g_edge_df['mst'], ['a', 'b']]
mst_edges = list(zip(mst_edges['a'], mst_edges['b']))
sp = ig.Graph([dim, dim, dim], edges=mst_edges)
shortest_path = sp.get_shortest_paths(0,(dim**3)-1, output="epath")

sp_edge_df = sp.get_edge_dataframe()
sp_edge_df['path'] = sp_edge_df.index.isin(shortest_path[0])
sp_edge_df = sp_edge_df.loc[sp_edge_df['path'],:]

sp_edge_df['ab'] = list(zip(sp_edge_df['source'], sp_edge_df['target']))

sp_verts = list(sp_edge_df['ab'])

def in_sp(v): 
    return v in sp_verts

g_edge_df['path'] = g_edge_df['ab'].apply(in_sp)

g_edge_df['a_xyz'] = g_edge_df['a'].apply(get_cartesian_coords)
g_edge_df['b_xyz'] = g_edge_df['b'].apply(get_cartesian_coords)
g_edge_df['ab_xyz'] = list(zip(g_edge_df['a_xyz'], g_edge_df['b_xyz']))

In [50]:
g_edge_df['color'] = "white"
g_edge_df.loc[g_edge_df['mst'], 'color'] = "purple"
g_edge_df.loc[g_edge_df['path'], 'color'] = "forestgreen"

g_vertex_df['vcolor'] = "purple"
g_vertex_df.loc[[0,len(g_vertex_df)-1], 'vcolor'] = "forestgreen"
g_vertex_df['vsize'] = 10
g_vertex_df.loc[[0,len(g_vertex_df)-1], 'vsize'] = 50

wall_color = "grey"
wall_edge_color = "black"

In [51]:
def wall_verts(v):
    if v[0][0] != v[1][0]:
        x = [(v[0][0] + v[1][0])/2] * 4
        y = [v[0][1] - .5, v[1][1] + .5]
        y=y+y[::-1]
        z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
    elif v[0][1] != v[1][1]:
        x= [v[0][0] - .5, v[1][0] + .5]
        x=x+x[::-1]
        y = [(v[0][1] + v[1][1])/2]*4
        z = [v[0][2] + .5]*2 + [v[1][2] - .5]*2
    else:
        x= [v[0][0] - .5, v[1][0] + .5]
        x=x+x[::-1]
        y = [v[0][1] - .5]*2 + [v[1][1] + .5]*2     
        z = [(v[0][2] + v[1][2])/2]*4
        
    #coord = np.array(np.meshgrid(x,y,z)).T.reshape(-1,3)
    #coord = list(map(tuple,coord))
    
    return list(zip(x,y,z))

In [52]:
g_edge_df['wall'] = g_edge_df['ab_xyz'].apply(wall_verts)

wall_df = g_edge_df.loc[g_edge_df['mst']==False, 'wall']

In [53]:
fig = plt.figure(("Hyper Maze " + str(dim) + "x" + str(dim)))
ax = fig.add_subplot(projection='3d')

ax.set_axis_off()
plt.tight_layout()

ax.scatter(g_vertex_df['x'], g_vertex_df['y'], g_vertex_df['z'], 
           c=g_vertex_df['vcolor'], 
           s=g_vertex_df['vsize'])

ax.add_collection3d(Line3DCollection(np.array(g_edge_df.loc[g_edge_df['mst'],'ab_xyz']),
                                     colors=g_edge_df.loc[g_edge_df['mst'],'color']))

walls = ax.add_collection3d(Poly3DCollection(list(wall_df), linewidths=1, alpha=0.2))
walls.set_facecolor(wall_color)
walls.set_edgecolor(wall_edge_color)
ax.add_collection3d(walls)

ax.set_xlim([-(dim),(dim)])
ax.set_ylim([-(dim),(dim)])
ax.set_zlim([-(dim),(dim)])

(-4.0, 4.0)